In [1]:
!CUDA_LAUNCH_BLOCKING=1

In [2]:
import os
import pandas
import numpy as np


In [3]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [4]:
wic_path  = "/content/drive/My Drive/datasets/WiC_dataset"


In [5]:
! ls "/content/drive/My Drive/datasets/WiC_dataset"

dev  README.txt  test  train


In [6]:
wic_train_path_d = os.path.join(wic_path, "train/train.data.txt")
wic_train_path_g = os.path.join(wic_path, "train/train.gold.txt")

train_data_df = pandas.read_csv(wic_train_path_d, sep = "\t", names = ['word', 'pos', 'position1', 'sent1', 'sent2'])
train_gold_df = pandas.read_csv(wic_train_path_g, sep = "\t", names = ['label'])

train_data_df[['position1', 'position2']] = train_data_df['position1'].str.split("-", expand = True)
train_data_df = train_data_df[['word', 'pos', 'position1', 'position2', 'sent1', 'sent2']]

train_data_df.head()
# train_gold_df.head()

,word,pos,position1,position2,sent1,sent2
0,carry,V,2,1,You must carry your camping gear .,Sound carries well over water .
1,go,V,2,6,Messages must go through diplomatic channels .,Do you think the sofa will go through the door ?
2,break,V,0,2,Break an alibi .,The wholesaler broke the container loads into ...
3,cup,N,8,4,He wore a jock strap with a metal cup .,Bees filled the waxen cups with honey .
4,academy,N,1,2,The Academy of Music .,The French Academy .


In [7]:
# do same for dev data
wic_dev_path_d = os.path.join(wic_path, "dev/dev.data.txt")
wic_dev_path_g = os.path.join(wic_path, "dev/dev.gold.txt")

dev_data_df = pandas.read_csv(wic_dev_path_d, sep = "\t", names = ['word', 'pos', 'position1', 'sent1', 'sent2'])
dev_gold_df = pandas.read_csv(wic_dev_path_g, sep = "\t", names = ['label'])

dev_data_df[['position1', 'position2']] = dev_data_df['position1'].str.split("-", expand = True)
dev_data_df = dev_data_df[['word', 'pos', 'position1', 'position2', 'sent1', 'sent2']]

dev_data_df.head()
# dev_gold_df.head()

,word,pos,position1,position2,sent1,sent2
0,board,N,2,2,Room and board .,He nailed boards across the windows .
1,circulate,V,0,4,Circulate a rumor .,This letter is being circulated among the facu...
2,hook,V,0,1,Hook a fish .,"He hooked a snake accidentally , and was so sc..."
3,recreation,N,1,9,For recreation he wrote poetry and solved cros...,Drug abuse is often regarded as a form of recr...
4,domesticity,N,4,6,Making a hobby of domesticity .,A royal family living in unpretentious domesti...


In [8]:
!nvidia-smi

Tue Oct 13 08:54:47 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [10]:
!pip install transformers

     |████████████████████████████████| 1.1MB 2.6MB/s 
     |████████████████████████████████| 1.1MB 8.2MB/s 
     |████████████████████████████████| 890kB 17.3MB/s 
     |████████████████████████████████| 3.0MB 22.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=6b3d75db0757ae4ce8dbe6671b976bf8d4ef7f84ddbcd2c214e22ac4087907ce
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [11]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [12]:
# sentences is a list 0f str
train_sentences_1 =  list(train_data_df['sent1'][:])
train_sentences_2 = list(train_data_df['sent2'][:])
print(len(train_sentences_1))

dev_sentences_1 =  list(dev_data_df['sent1'][:])
dev_sentences_2 = list(dev_data_df['sent2'][:])
print(len(dev_sentences_1))

5428
638


In [13]:
encoded_inputs_train = tokenizer(train_sentences_1, train_sentences_1, padding = True, truncation = True, return_tensors = 'pt')
print(len(encoded_inputs_train['input_ids'][0]))
print(type(encoded_inputs_train['input_ids']))

encoded_inputs_dev = tokenizer(dev_sentences_1, dev_sentences_2, padding = True, truncation = True, return_tensors = 'pt')
print(len(encoded_inputs_dev['input_ids'][0]))
print(type(encoded_inputs_dev['input_ids']))

75
<class 'torch.Tensor'>
60
<class 'torch.Tensor'>


In [14]:
# labels = torch.from_numpy(train_gold_df['label'].values)
train_gold_df_tmp = train_gold_df.replace({'F' : 0, 'T' : 1})
train_labels = torch.from_numpy(train_gold_df_tmp.values)
print(train_labels)

dev_gold_df_tmp = dev_gold_df.replace({'F' : 0, 'T' : 1})
dev_labels = torch.from_numpy(dev_gold_df_tmp.values)
print(len(dev_labels))


pos1 = torch.from_numpy(train_data_df['position1'])
pos2 = torch.from_numpy(train_data_df['position2'])
print(pos1)

tensor([[0],
        [0],
        [0],
        ...,
        [1],
        [1],
        [1]])
638


In [15]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(encoded_inputs_train['input_ids'], encoded_inputs_train['token_type_ids'],
                              encoded_inputs_train['attention_mask'], train_labels,)

dev_dataset = TensorDataset(encoded_inputs_dev['input_ids'], encoded_inputs_dev['token_type_ids'],
                              encoded_inputs_dev['attention_mask'], dev_labels)


In [34]:
print(encoded_inputs_train['input_ids'].size())
print(encoded_inputs_train['input_ids'][3])
print(340*16)

torch.Size([5428, 75])
tensor([  101,  2002,  5078,  1037, 23407, 16195,  2007,  1037,  3384,  2452,
         1012,   102,  2002,  5078,  1037, 23407, 16195,  2007,  1037,  3384,
         2452,  1012,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0])
5440


In [92]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 8

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
dev_dataloader = DataLoader(
            dev_dataset, # The validation samples.
            sampler = SequentialSampler(dev_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

**CUTOFF**

In [118]:
from transformers import BertForNextSentencePrediction, AdamW, BertConfig
import torch.nn as nn

import torch.nn as nn
import copy
import torch
from transformers import BertTokenizer, BertModel

class Berthead(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.embeddings = nn.Embedding(30522, 768, padding_idx=0)
        self.embeddings.weight = copy.deepcopy(self.bert.embeddings.word_embeddings.weight)
        print(self.embeddings)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None, position_ids=None, head_mask=None):
        outputs = self.bert(input_ids, position_ids=position_ids, token_type_ids=token_type_ids,
                            attention_mask=attention_mask, head_mask=head_mask)
        # get_output_embeddings
        
        return self.embeddings(input_ids)

        # pooled_output = outputs[1] # see note below

        # pooled_output = self.dropout(pooled_output)
        # logits = self.classifier(pooled_output)

        # outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

        # if labels is not None:
        #     if self.num_labels == 1:
        #         #  We are doing regression
        #         loss_fct = MSELoss()
        #         loss = loss_fct(logits.view(-1), labels.view(-1))
        #     else:
        #         loss_fct = CrossEntropyLoss()
        #         loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
        #     outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)

model = Berthead().cuda()

Embedding(30522, 768, padding_idx=0)


In [125]:
# print(b_input_ids.size())
print(b_attention_mask[0])
print(b_token_type_ids[0])
# embeddings  = model(b_input_ids, b_token_type_ids, b_attention_mask)
# print(embeddings.size())
# print(embeddings[0][0])


tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0], device='cuda:0')


In [123]:
print(outputs[0][0])

# print(len(outputs))
# print(outputs[0].size())
# print(outputs[1].size())
# # print(outputs[0])
# # from transformers import BertTokenizer, BertModel
# # tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# # model = BertModel.from_pretrained("bert-base-uncased")
# # text = "Replace me by any text you'd like."
# # encoded_input = tokenizer(text, return_tensors='pt')
# # output = model(**encoded_input)
# print(len(output))
# print(output[0].size())
# print(output[1].size())

# print()

tensor([[-1.7189e-01,  6.6056e-01, -5.7630e-01,  ..., -4.5400e-01,
          5.6141e-01,  1.4085e-01],
        [ 7.2446e-02,  5.7505e-01, -1.3618e-01,  ..., -2.7317e-02,
          7.6051e-01, -7.4321e-01],
        [-1.5714e-01,  4.9836e-02, -5.8785e-01,  ..., -9.7910e-02,
         -2.9752e-03, -2.7736e-01],
        ...,
        [ 2.7471e-01, -5.2760e-02, -2.7542e-02,  ...,  5.4375e-01,
          9.3893e-02, -2.8732e-01],
        [ 5.2591e-02,  1.8826e-01,  2.3509e-01,  ...,  8.0838e-02,
         -1.7803e-01,  1.6981e-01],
        [ 3.1541e-01,  2.3600e-01,  2.7340e-01,  ..., -2.8787e-04,
          7.5056e-02, -2.5763e-02]], device='cuda:0', grad_fn=<SelectBackward>)


In [101]:
model = Berthead().to(device)
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

Embedding(30522, 768, padding_idx=0)
The BERT model has 200 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.atten

In [95]:
optimizer = AdamW(model.parameters(),
                  lr = 1e-5, # args.learning_rate - default is 5e-5, our notebook had 1e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                  )

# scheduler??


In [96]:
def loss_fn(output, targets):
  return nn.BCEWithLogitsLoss()(output, targets)

In [97]:
# from transformers import get_linear_schedule_with_warmup

# # Number of training epochs. The BERT authors recommend between 2 and 4. 
# # We chose to run for 4, but we'll see later that this may be over-fitting the
# # training data.
# epochs = 4

# # Total number of training steps is [number of batches] x [number of epochs]. 
# # (Note that this is not the same as the number of training samples).
# total_steps = len(train_dataloader) * epochs

# # Create the learning rate scheduler.
# scheduler = get_linear_schedule_with_warmup(optimizer, 
#                                             num_warmup_steps = 0, # Default value in run_glue.py
#                                             num_training_steps = total_steps)

In [98]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [99]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [84]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128


# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()
epochs = 5
# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 5 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.

        b_input_ids = batch[0].to(device)
        b_token_type_ids = batch[1].to(device)
        b_attention_mask = batch[2].to(device)
        b_labels = batch[3].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        loss, logits = model(b_input_ids, b_token_type_ids, b_attention_mask, b_labels)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        
        # print(loss)
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # # Update the learning rate.
        # scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    t_labels = []
    t_preds = []

    # Evaluate data for one epoch
    for batch in dev_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.

        b_input_ids = batch[0].to(device)
        b_token_type_ids = batch[1].to(device)
        b_attention_mask = batch[2].to(device)
        b_labels = batch[3].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            loss, logits = model(b_input_ids, b_token_type_ids, b_attention_mask, b_labels)
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.cpu().detach().numpy()
        label_ids = b_labels.cpu().numpy()

        t_labels.append(label_ids)
        t_preds.append(logits)
        

    all_dev_labels = np.concatenate(t_labels, axis=0)
    all_dev_logits = np.concatenate(t_preds, axis=0)
    # Report the final accuracy for this validation run.
    avg_val_accuracy = flat_accuracy(all_dev_logits, all_dev_labels)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(dev_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 5 ========
Training...


TypeError: ignored

In [77]:
from sklearn import metrics
# print()
preds = np.argmax(all_dev_logits, axis=1).flatten()
logits = all_dev_labels.flatten()
# print(all_dev_labels.flatten())
print(np.count_nonzero(logits))
cm=metrics.confusion_matrix(preds, logits)
print(cm)
# plt.imshow(cm, cmap='binary')

319
[[188 169]
 [131 150]]


Embedding(30522, 768, padding_idx=0)


In [ ]:
# BERTforSequence classficiation
# overfitting after 5 epochs
# Number of epochs  - 5, Batchsize 16,  Validation accuracy: 61 
# Number of epochs - 5, Batchsize 32 VA: 48 , Trainling loss reduced to 37. Overfitting (Early stopped) CM - nothing interesting
# Number of epochs  - 5, Batchsize 8,  Validation accuracy: 58 

# BERTforNSP
# Number of epochs  - 10, Batchsize 16,  Validation accuracy: 61
# Number of epochs - 10, Batchsize 32 VA: 48 , Trainling loss reduced to 37. Overfitting (Early stopped) CM - nothing interesting
# Number of epochs  - 10, Batchsize 8,  Validation accuracy: 61

#attention mask on word
#Neural Network with CLS
#Neural Network with embeddings
#Neural Network with embeddings and POS